<a href="https://colab.research.google.com/github/FuturisticLover123/labs/blob/main/lab8/lab8solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [253]:
from sklearn.metrics import accuracy_score

class NBC:
  def __init__(self):
    pass

  def pxy(self, distribution, x, kwargs):
    if distribution == "gaussian":
      m, q = kwargs
      return (1 / (q * np.sqrt(2 * np.pi)) ) * np.exp(-((x - m)**2 / (2 * q**2)))

    if distribution == "bernoulli":
      pass


  def dist_data(self,distribution, x, y):
    if distribution == "gaussian":
      return np.mean(x), np.std(x)

    if distribution == "bernoulli":
      return np.mean(y), 1 - mean(y)

  def fit(self, x, y, feature_distribution: dict = {}):
    self.x, self.y = x, y
    self.classes = list(set(self.y))
    self.feature_distribution = feature_distribution
    index_class = { clas: np.where(self.y == clas)[0] for clas in self.classes }
    self.prior_probability = { clas: len(index_class[clas]) / len(self.y) for clas in self.classes }

    self.classifying_data = dict()
    for feature in range(x.shape[1]):
      for clas in self.classes:
        distribution = feature_distribution.get(feature, "gaussian")
        self.classifying_data[clas] = self.classifying_data.get(clas, dict())
        self.classifying_data[clas][feature] = self.dist_data(distribution, x[index_class[clas], feature], y[index_class[clas]])


    return self

  def classify(self, x):
    weights = { clas: np.log(self.prior_probability[clas]) for clas in self.classes }
    for clas in self.classes:
      for feature in range(x.shape[0]):
        distribution = self.feature_distribution.get(feature, "gaussian")
        weights[clas] += np.log(self.pxy(distribution, x[feature] ,self.classifying_data[clas][feature]))

    return max(weights, key=weights.get)

  def predict(self, x):
    return np.array([self.classify(x_i) for x_i in x])

  def score(self, x, y):
    return accuracy_score(self.predict(x), y)

  def get_params(self, deep=True):
    return dict()


  def set_params(self, **params):
    for key, value in params.items():
        setattr(self, key, value)
    return self




In [254]:
nbc = NBC().fit(x,y)

In [255]:
nbc.score(x,y)

1.0

In [256]:
from sklearn.datasets import make_classification


In [257]:
x,y = make_classification(n_samples = 1000,n_features = 30,n_redundant=20)

In [258]:
from sklearn.datasets import load_iris

In [259]:
data = load_iris()
x = data["data"]
y = data["target"]

In [260]:
nbc=NBC()
nbc.fit(x,y)
nbc.score(x,y)

0.96

In [261]:
#task2

In [262]:
import pandas as pd
from sklearn.model_selection import cross_val_score

In [263]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [231]:
!kaggle datasets download -d valakhorasani/mobile-device-usage-and-user-behavior-dataset -p /content/sample_data/lab8
!unzip /content/sample_data/lab8/mobile-device-usage-and-user-behavior-dataset.zip -d /content/sample_data/lab8

Dataset URL: https://www.kaggle.com/datasets/valakhorasani/mobile-device-usage-and-user-behavior-dataset
License(s): apache-2.0
mobile-device-usage-and-user-behavior-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/sample_data/lab8/mobile-device-usage-and-user-behavior-dataset.zip
replace /content/sample_data/lab8/user_behavior_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [264]:
data = pd.read_csv("/content/sample_data/lab8/user_behavior_dataset.csv").dropna()

In [265]:
data["Device Model"] = data["Device Model"].apply(lambda x: x.split(" ")[0])

In [266]:
x = pd.get_dummies(data, columns = ["Device Model", "Operating System", "Gender"]).astype(float).drop(columns = ["User ID", "User Behavior Class"]).values
y = data["User Behavior Class"].values

In [267]:
model = NBC()
cross_val_score(estimator=model, X=x, y=y, cv=10, n_jobs=-1).mean()

1.0

In [268]:
#task3

In [284]:
import pandas as pd

data = pd.read_csv('/content/sample_data/lab8/spam.csv', encoding='windows-1251')
data = data.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])


In [285]:
data.head(3)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [272]:
!pip install nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
import nltk




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [286]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)

    words = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    return ' '.join(words)


text = "I running love programming in Python, and I enjoy solving coding challenges!"

processed_text = preprocess_text(text)
print(processed_text)


run love program python enjoy solv code challeng


In [287]:
data["v2"] = data["v2"].apply(preprocess_text)
data["v1"] = data['v1'].map({'ham': 0, 'spam': 1})

In [288]:
data

,v1,v2
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri 2 wkli comp win fa cup final tkt 21...
3,0,u dun say earli hor u c alreadi say
4,0,nah dont think go usf life around though
...,...,...
5567,1,2nd time tri 2 contact u u еј750 pound prize 2...
5568,0,м_ b go esplanad fr home
5569,0,piti mood soani suggest
5570,0,guy bitch act like id interest buy someth els ...


In [289]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

# Применяем векторизацию к тексту
X = vectorizer.fit_transform(data['v2'])

# Преобразуем полученную матрицу в DataFrame
word_freq = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Добавляем метки (spam/ham) в DataFrame
word_freq['v1'] = data['v1']

# Разделяем данные по меткам
spam_words = word_freq[word_freq['v1'] == 0].drop('v1', axis=1)
ham_words = word_freq[word_freq['v1'] == 1].drop('v1', axis=1)

# Суммируем частоты слов для спама и не спама
spam_word_freq = spam_words.sum(axis=0).sort_values(ascending=False)
ham_word_freq = ham_words.sum(axis=0).sort_values(ascending=False)

# Создаем итоговую таблицу частот для слов в зависимости от категории
result = pd.DataFrame({
    'spam_freq': spam_word_freq,
    'ham_freq': ham_word_freq
}).fillna(0)  # Если какое-то слово встречается только в одной категории

# Выводим результат
print(result)


              spam_freq  ham_freq
008704050406          0         2
0089mi                0         1
0121                  0         1
01223585236           0         1
01223585334           0         2
...                 ...       ...
ы_                   14         0
ы_thank               1         0
ып                    0         1
ыпharri               0         1
ыт                    9         0

[7914 rows x 2 columns]


In [321]:
class SpamNBC(NBC):
  def __init__(self, alpha=1):
    super().__init__()
    self.alpha = alpha

  def fit(self, x, y):
    self.x, self.y = x, y
    self.classes = list(set(self.y))
    self.classifying_data = dict()
    indexes = { clas: np.where(self.y == clas)[0] for clas in self.classes }
    self.prior_probability = { clas: len(indexes[clas]) / len(y) for clas in self.classes }
    self.class_sizes = { clas: len(indexes[clas]) for clas in self.classes }
    total_words = dict()

    for clas in self.classes:
      self.classifying_data[clas] = dict()
      for x_i in self.x[indexes[clas]]:
        total_words[clas] = total_words.get(clas, 0) + len(x_i.split())
        for word in x_i.split():
          self.classifying_data[clas][word] = self.classifying_data[clas].get(word, 0) + 1
    return self

  def classify(self, x):
    weights = {clas: np.log(self.prior_probability[clas]) for clas in self.classes }
    for clas in self.classes:
      for word in x.split():
        weights[clas] += np.log((self.alpha + self.classifying_data[clas].get(word, 0)) / (self.class_sizes[clas] + self.alpha * 1000))
    return max(weights, key=weights.get)





In [322]:
Spambc = SpamNBC()
Spambc.fit(x, y)

In [326]:
Spambc.score(x, y)

0.9813352476669059

In [ ]:
#фильмы

In [329]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p /content/sample_data/lab8;
!unzip /content/sample_data/lab8/imdb-dataset-of-50k-movie-reviews.zip -d /content/sample_data/lab8;


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 58% 15.0M/25.7M [00:00<00:00, 154MB/s]
100% 25.7M/25.7M [00:00<00:00, 141MB/s]
Archive:  /content/sample_data/lab8/imdb-dataset-of-50k-movie-reviews.zip
  inflating: /content/sample_data/lab8/IMDB Dataset.csv  


In [330]:
data = pd.read_csv('/content/sample_data/lab8/IMDB Dataset.csv')

In [334]:
data["review"] = data["review"].apply(preprocess_text)
data["sentiment"] = data['sentiment'].map({'positive': 0, 'negative': 1})

In [347]:
x = data["review"].values
y = data["sentiment"].values

In [348]:
Spambc = SpamNBC()
Spambc.fit(x, y)

In [350]:
Spambc.score(x,y)

0.90428